# Counterfactuals: orcs

## Prepare data

In [1]:
from training.creating_dataset import load_data, preprocess_data

In [2]:
import pandas as pd
import numpy as np
import sys

import os
import pathlib

sys.path.append("../..")

current_path = os.getcwd()
DATASETS_DIR = pathlib.Path(current_path).parent.parent / "pathfinder_2e_data"
DATASET_FILES = [
    "pathfinder-bestiary.db",
    "pathfinder-bestiary-2.db",
    "pathfinder-bestiary-3.db",
]
DATASET_PATHS = [f"{DATASETS_DIR}/{file}" for file in DATASET_FILES]

df = load_data(DATASET_PATHS)

In [3]:
# df = df[ORDERED_CHARACTERISTICS + ["book", "level"]]
df.head()

,_id,img,items,name,type,system.abilities.cha.mod,system.abilities.con.mod,system.abilities.dex.mod,system.abilities.int.mod,system.abilities.str.mod,...,system.traits.size.value,system.traits.value,system.schema.version,system.schema.lastMigration,flags.core.sourceId,system.attributes.resistances,system.traits.attitude.value,prototypeToken.name,system.attributes.hardness.value,system.attributes.adjustment
0,024PqcF8yMRBrPuq,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'B7rzf0nBJmzg8x0y', 'img': 'systems/p...",Adult White Dragon,npc,1,5,2,1,7,...,lg,"[cold, dragon]",0.827,None,Compendium.pf2e.pathfinder-bestiary.024PqcF8yM...,NaN,NaN,NaN,NaN,NaN
1,05E3kkjoLZVjFOeO,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'mYNDkS27YQRyy1F0', 'img': 'systems/p...",Brontosaurus,npc,1,5,0,-4,9,...,grg,"[animal, dinosaur]",0.827,None,Compendium.pf2e.pathfinder-bestiary.05E3kkjoLZ...,NaN,NaN,NaN,NaN,NaN
2,05wwpHHsBlxBbdkN,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'WkPeg600zGONsuJz', 'img': 'systems/p...",Giant Anaconda,npc,-2,6,3,-4,7,...,huge,[animal],0.827,None,Compendium.pf2e.pathfinder-bestiary.05wwpHHsBl...,NaN,NaN,NaN,NaN,NaN
3,0FGz2eXm0SB04sJW,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'joBYS96mXSnZC1WB', 'img': 'systems/p...",Ancient Green Dragon,npc,6,5,4,6,7,...,grg,"[amphibious, dragon]",0.827,None,Compendium.pf2e.pathfinder-bestiary.0FGz2eXm0S...,NaN,NaN,NaN,NaN,NaN
4,0HjVFx8qIKDCfblg,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'xlUnQcjrxeMXu5Cd', 'img': 'systems/p...",Lantern Archon,npc,1,1,3,-1,-5,...,sm,"[archon, celestial]",0.827,None,Compendium.pf2e.pathfinder-bestiary.0HjVFx8qIK...,"[{'type': 'fire', 'value': 3}]",NaN,NaN,NaN,NaN


In [18]:
df_orcs = df[df.name.str.contains("Orc ")]

In [19]:
df_orcs.name

240    Orc Warchief
284       Orc Brute
323     Orc Warrior
Name: name, dtype: object

In [47]:
orcs_ids_names = {df_orcs.loc[idx]["name"]: idx for idx in df_orcs.index}
orcs_ids_names

{'Orc Warchief': 240, 'Orc Brute': 284, 'Orc Warrior': 323}

In [48]:
from serving.backend.constants import ORDERED_CHARACTERISTICS


orcs = preprocess_data(df_orcs, ORDERED_CHARACTERISTICS)
orcs

,con,level,book,dex,hp,str,ac,cha,wis,int
240,1,2,Pathfinder Bestiary,2,32,4,19,2,1,-1
284,3,0,Pathfinder Bestiary,2,15,3,15,0,1,-1
323,3,1,Pathfinder Bestiary,2,23,4,18,0,1,-1


In [52]:
orcs = orcs[ORDERED_CHARACTERISTICS + ["level"]]

In [53]:
orcs

,cha,con,dex,int,str,wis,ac,hp,level
240,2,1,2,-1,4,1,19,32,2
284,0,3,2,-1,3,1,15,15,0
323,0,3,2,-1,4,1,18,23,1


In [44]:
dataset = preprocess_data(df, ORDERED_CHARACTERISTICS)
dataset = dataset[ORDERED_CHARACTERISTICS + ["level"]]

In [45]:
dataset.head()

,cha,con,dex,int,str,wis,ac,hp,level
0,1,5,2,1,7,2,29,215,10
1,1,5,0,-4,9,2,28,220,10
2,-2,6,3,-4,7,3,25,175,8
3,6,5,4,6,7,5,41,315,17
4,1,1,3,-1,-5,1,16,20,1


In [57]:
import joblib


model = joblib.load(filename="../../saved_models/current_model.pkl")

## Counterfactuals for orcs

In [46]:
from serving.backend.generate_counterfactuals import generate_counterfactuals

In [66]:
orc_brute = orcs.loc[orcs_ids_names["Orc Brute"]].to_dict()
orc_warrior = orcs.loc[orcs_ids_names["Orc Warrior"]].to_dict()
orc_warchief = orcs.loc[orcs_ids_names["Orc Warchief"]].to_dict()

### Orc Brute

Compare to others (levels 1 and 2)

In [146]:
result = generate_counterfactuals(
    monster_stats=orc_brute, model=model, new_level=1, df=dataset, total_cf=15
)

100%|██████████| 1/1 [00:08<00:00,  8.18s/it]


In [147]:
diff_columns = [f"{col}_diff" for col in ORDERED_CHARACTERISTICS]


def compare_monsters(cfs, compare_to_monster):
    compare_df = pd.DataFrame(columns=ORDERED_CHARACTERISTICS)
    diff_df = pd.DataFrame(columns=diff_columns)

    for cf in cfs:
        cfs_comp_dict = dict()
        diff_dict = dict()
        for i, col in enumerate(ORDERED_CHARACTERISTICS):
            val = cf[i]
            cfs_comp_dict[col] = [val] if val != compare_to_monster[col] else ["-"]
            diff_dict[f"{col}_diff"] = [compare_to_monster[col] - val]

        comparison = pd.DataFrame.from_dict(cfs_comp_dict)
        difference = pd.DataFrame.from_dict(diff_dict)
        compare_df = pd.concat([comparison, compare_df.loc[:]]).reset_index(drop=True)
        diff_df = pd.concat([difference, diff_df]).reset_index(drop=True)

    return compare_df, diff_df

In [148]:
comp, diff = compare_monsters(result["values"], orc_warrior)

In [149]:
comp

,cha,con,dex,int,str,wis,ac,hp
0,-,4.0,4.0,-4.0,3.0,-,16.0,24.0
1,-,-,4.0,-,2.0,2.0,17.0,20.0
2,-2.0,-,3.0,-5.0,3.0,-,16.0,20.0
3,-2.0,-,3.0,-4.0,3.0,-,16.0,20.0
4,-1.0,-,3.0,-4.0,3.0,-,8.0,20.0
5,-1.0,2.0,-,-4.0,3.0,-,17.0,17.0
6,1.0,2.0,-,0.0,-,-,15.0,24.0
7,-,-,-,0.0,2.0,2.0,14.0,20.0
8,-5.0,-,-,-,-5.0,-,16.0,14.0
9,-,-,4.0,-4.0,3.0,-,16.0,21.0


In [150]:
diff

,cha_diff,con_diff,dex_diff,int_diff,str_diff,wis_diff,ac_diff,hp_diff
0,0.0,-1.0,-2.0,3.0,1.0,0.0,2.0,-1.0
1,0.0,0.0,-2.0,0.0,2.0,-1.0,1.0,3.0
2,2.0,0.0,-1.0,4.0,1.0,0.0,2.0,3.0
3,2.0,0.0,-1.0,3.0,1.0,0.0,2.0,3.0
4,1.0,0.0,-1.0,3.0,1.0,0.0,10.0,3.0
5,1.0,1.0,0.0,3.0,1.0,0.0,1.0,6.0
6,-1.0,1.0,0.0,-1.0,0.0,0.0,3.0,-1.0
7,0.0,0.0,0.0,-1.0,2.0,-1.0,4.0,3.0
8,5.0,0.0,0.0,0.0,9.0,0.0,2.0,9.0
9,0.0,0.0,-2.0,3.0,1.0,0.0,2.0,2.0


In [151]:
result = generate_counterfactuals(
    monster_stats=orc_brute, model=model, new_level=2, df=dataset, total_cf=15
)

100%|██████████| 1/1 [00:03<00:00,  3.53s/it]


In [152]:
comp, diff = compare_monsters(result["values"], orc_warchief)

In [153]:
comp

,cha,con,dex,int,str,wis,ac,hp
0,0.0,3.0,3.0,-5.0,5.0,-,16.0,33.0
1,0.0,3.0,-,-5.0,-,0.0,16.0,35.0
2,0.0,2.0,3.0,-,-,-,18.0,29.0
3,0.0,3.0,-,-4.0,-,2.0,17.0,35.0
4,-1.0,3.0,-,0.0,-,-,18.0,34.0
5,-5.0,3.0,-,-,3.0,-2.0,17.0,24.0
6,0.0,3.0,4.0,-4.0,0.0,-,15.0,30.0
7,0.0,4.0,-,-,3.0,3.0,18.0,30.0
8,-5.0,3.0,-,-,3.0,0.0,17.0,34.0
9,0.0,2.0,4.0,-,3.0,-,-,28.0


In [154]:
diff

,cha_diff,con_diff,dex_diff,int_diff,str_diff,wis_diff,ac_diff,hp_diff
0,2.0,-2.0,-1.0,4.0,-1.0,0.0,3.0,-1.0
1,2.0,-2.0,0.0,4.0,0.0,1.0,3.0,-3.0
2,2.0,-1.0,-1.0,0.0,0.0,0.0,1.0,3.0
3,2.0,-2.0,0.0,3.0,0.0,-1.0,2.0,-3.0
4,3.0,-2.0,0.0,-1.0,0.0,0.0,1.0,-2.0
5,7.0,-2.0,0.0,0.0,1.0,3.0,2.0,8.0
6,2.0,-2.0,-2.0,3.0,4.0,0.0,4.0,2.0
7,2.0,-3.0,0.0,0.0,1.0,-2.0,1.0,2.0
8,7.0,-2.0,0.0,0.0,1.0,1.0,2.0,-2.0
9,2.0,-1.0,-2.0,0.0,1.0,0.0,0.0,4.0


### Orc Warrior

In [155]:
result = generate_counterfactuals(
    monster_stats=orc_warrior, model=model, new_level=0, df=dataset, total_cf=15
)

100%|██████████| 1/1 [01:27<00:00, 87.52s/it]


In [156]:
comp, diff = compare_monsters(result["values"], orc_brute)

In [157]:
comp

,cha,con,dex,int,str,wis,ac,hp
0,-,2.0,3.0,-4.0,-1.0,-,16.0,16.0
1,-,2.0,3.0,-,-,3.0,-,16.0
2,-,2.0,-,-5.0,-,2.0,16.0,-
3,-,8.0,-3.0,-2.0,6.0,-,18.0,11.0
4,-,-,1.0,0.0,2.0,2.0,16.0,-
5,-,10.0,-,-5.0,-,-,8.0,-
6,-,0.0,-,-,-,6.0,12.0,-
7,-,-,-3.0,-,10.0,-1.0,16.0,14.0
8,-,-,4.0,-,-5.0,-1.0,8.0,4.0
9,-,7.0,-,-2.0,-,-,9.0,18.0


#### Row **13**

**Here counterfactuals generated the same stats as *Orc brute* for *Orc warrior*** (change: 1 -> 0)

There is a chance that it is because *DiCE* was unable to genarate more than 13 different than monsters from dataset

In [158]:
diff

,cha_diff,con_diff,dex_diff,int_diff,str_diff,wis_diff,ac_diff,hp_diff
0,0.0,1.0,-1.0,3.0,4.0,0.0,-1.0,-1.0
1,0.0,1.0,-1.0,0.0,0.0,-2.0,0.0,-1.0
2,0.0,1.0,0.0,4.0,0.0,-1.0,-1.0,0.0
3,0.0,-5.0,5.0,1.0,-3.0,0.0,-3.0,4.0
4,0.0,0.0,1.0,-1.0,1.0,-1.0,-1.0,0.0
5,0.0,-7.0,0.0,4.0,0.0,0.0,7.0,0.0
6,0.0,3.0,0.0,0.0,0.0,-5.0,3.0,0.0
7,0.0,0.0,5.0,0.0,-7.0,2.0,-1.0,1.0
8,0.0,0.0,-2.0,0.0,8.0,2.0,7.0,11.0
9,0.0,-4.0,0.0,1.0,0.0,0.0,6.0,-3.0


In [159]:
result = generate_counterfactuals(
    monster_stats=orc_warrior, model=model, new_level=2, df=dataset, total_cf=15
)

100%|██████████| 1/1 [00:06<00:00,  6.45s/it]


In [160]:
comp, diff = compare_monsters(result["values"], orc_warchief)

In [161]:
comp

,cha,con,dex,int,str,wis,ac,hp
0,0.0,4.0,-,-4.0,-,2.0,18.0,-
1,0.0,3.0,1.0,-3.0,-,2.0,18.0,30.0
2,-1.0,3.0,3.0,-5.0,-,-,18.0,34.0
3,-1.0,3.0,1.0,-5.0,-,-,18.0,30.0
4,-2.0,3.0,-,-5.0,3.0,-,18.0,30.0
5,1.0,3.0,1.0,-,-,2.0,18.0,28.0
6,-2.0,3.0,-,-4.0,3.0,-,18.0,-
7,-2.0,3.0,-,-4.0,3.0,-,18.0,30.0
8,0.0,3.0,-,-4.0,-,2.0,17.0,35.0
9,0.0,3.0,4.0,0.0,-,-,-,30.0


In [162]:
diff

,cha_diff,con_diff,dex_diff,int_diff,str_diff,wis_diff,ac_diff,hp_diff
0,2.0,-3.0,0.0,3.0,0.0,-1.0,1.0,0.0
1,2.0,-2.0,1.0,2.0,0.0,-1.0,1.0,2.0
2,3.0,-2.0,-1.0,4.0,0.0,0.0,1.0,-2.0
3,3.0,-2.0,1.0,4.0,0.0,0.0,1.0,2.0
4,4.0,-2.0,0.0,4.0,1.0,0.0,1.0,2.0
5,1.0,-2.0,1.0,0.0,0.0,-1.0,1.0,4.0
6,4.0,-2.0,0.0,3.0,1.0,0.0,1.0,0.0
7,4.0,-2.0,0.0,3.0,1.0,0.0,1.0,2.0
8,2.0,-2.0,0.0,3.0,0.0,-1.0,2.0,-3.0
9,2.0,-2.0,-2.0,-1.0,0.0,0.0,0.0,2.0


### Orc Warchief

In [163]:
result = generate_counterfactuals(
    monster_stats=orc_warchief, model=model, new_level=0, df=dataset, total_cf=15
)

100%|██████████| 1/1 [01:11<00:00, 71.77s/it]


In [164]:
comp, diff = compare_monsters(result["values"], orc_brute)

In [165]:
comp

,cha,con,dex,int,str,wis,ac,hp
0,-,4.0,-,2.0,4.0,3.0,11.0,13.0
1,6.0,1.0,-,5.0,6.0,4.0,18.0,6.0
2,-,4.0,-,-4.0,4.0,3.0,16.0,-
3,-3.0,1.0,3.0,-4.0,-2.0,-,14.0,14.0
4,-3.0,1.0,3.0,-4.0,0.0,-,14.0,14.0
5,-,1.0,3.0,0.0,-5.0,-,18.0,10.0
6,-2.0,1.0,3.0,-4.0,0.0,-,13.0,12.0
7,-2.0,1.0,3.0,-4.0,0.0,-,16.0,-
8,-,1.0,3.0,-4.0,-1.0,-,16.0,14.0
9,6.0,7.0,5.0,2.0,4.0,-,19.0,4.0


#### Row **13**

**Here counterfactuals generated the same stats as *Orc warchief* for *Orc brute*** (change: 2 -> 0)

In [166]:
diff

,cha_diff,con_diff,dex_diff,int_diff,str_diff,wis_diff,ac_diff,hp_diff
0,0.0,-1.0,0.0,-3.0,-1.0,-2.0,4.0,2.0
1,-6.0,2.0,0.0,-6.0,-3.0,-3.0,-3.0,9.0
2,0.0,-1.0,0.0,3.0,-1.0,-2.0,-1.0,0.0
3,3.0,2.0,-1.0,3.0,5.0,0.0,1.0,1.0
4,3.0,2.0,-1.0,3.0,3.0,0.0,1.0,1.0
5,0.0,2.0,-1.0,-1.0,8.0,0.0,-3.0,5.0
6,2.0,2.0,-1.0,3.0,3.0,0.0,2.0,3.0
7,2.0,2.0,-1.0,3.0,3.0,0.0,-1.0,0.0
8,0.0,2.0,-1.0,3.0,4.0,0.0,-1.0,1.0
9,-6.0,-4.0,-3.0,-3.0,-1.0,0.0,-4.0,11.0


In [167]:
result = generate_counterfactuals(
    monster_stats=orc_warchief, model=model, new_level=1, df=dataset, total_cf=15
)

100%|██████████| 1/1 [00:08<00:00,  8.72s/it]


In [168]:
comp, diff = compare_monsters(result["values"], orc_warrior)

In [169]:
comp

,cha,con,dex,int,str,wis,ac,hp
0,-1.0,0.0,-,-4.0,3.0,-,16.0,17.0
1,-1.0,2.0,-,-4.0,3.0,-,17.0,17.0
2,4.0,1.0,3.0,0.0,0.0,-,16.0,25.0
3,2.0,2.0,3.0,-,3.0,0.0,17.0,18.0
4,3.0,1.0,1.0,-,2.0,2.0,15.0,13.0
5,1.0,1.0,4.0,0.0,2.0,-,-,19.0
6,-1.0,1.0,4.0,-,-,-2.0,17.0,16.0
7,-2.0,1.0,-,-,3.0,-2.0,15.0,24.0
8,2.0,4.0,3.0,-5.0,-,-,16.0,25.0
9,2.0,2.0,3.0,-,0.0,-,16.0,22.0


In [170]:
diff

,cha_diff,con_diff,dex_diff,int_diff,str_diff,wis_diff,ac_diff,hp_diff
0,1.0,3.0,0.0,3.0,1.0,0.0,2.0,6.0
1,1.0,1.0,0.0,3.0,1.0,0.0,1.0,6.0
2,-4.0,2.0,-1.0,-1.0,4.0,0.0,2.0,-2.0
3,-2.0,1.0,-1.0,0.0,1.0,1.0,1.0,5.0
4,-3.0,2.0,1.0,0.0,2.0,-1.0,3.0,10.0
5,-1.0,2.0,-2.0,-1.0,2.0,0.0,0.0,4.0
6,1.0,2.0,-2.0,0.0,0.0,3.0,1.0,7.0
7,2.0,2.0,0.0,0.0,1.0,3.0,3.0,-1.0
8,-2.0,-1.0,-1.0,4.0,0.0,0.0,2.0,-2.0
9,-2.0,1.0,-1.0,0.0,4.0,0.0,2.0,1.0


## Summary

* **Orc brute**:
    * There were only 2 cases (level change: 0 -> 2) when *DiCE* decided to not change one of: *ac*, *hp*
    * In some cases to increase level it was suggested to choose *hp* that's value is smaller than orc warrior/warchief: mostly by 1, 2, 3 but there was a case where 7 was suggested
    * It was always suggested to chose ac with larger value than orc warrior/warchief (except 2 mentioned cases)
    * In case 0->2 *DiCE* always suggested changing 2 first characteristics: *cha*, *con*
    * In case 0->1 options were more diverse but suggestions usually chose different values for: *int*, *str*